<a href="https://colab.research.google.com/github/joejo-joestar/Agentic-AI-Notebooks/blob/main/colab/01_react_agent_demos/01_react_agent_country_deets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 - ReAct Country Details Agent Example

## Requirements
These are the required _Libraries_ and _Environment Variables_ for this notebook


### Libraries Required


- [LangSmith](https://docs.langchain.com/langsmith/home)
- [LangChain `1.0.3+`](https://reference.langchain.com/python/langchain/langchain/)
- [LangChain Core](https://reference.langchain.com/python/langchain_core/)
- [LangChain Groq Model Provider](https://reference.langchain.com/python/integrations/langchain_groq/)

In [ ]:
%pip install --upgrade --force-reinstall langsmith
%pip install --upgrade --force-reinstall langchain
%pip install --upgrade --force-reinstall langchain-core

%pip install --upgrade --force-reinstall langchain-groq


In [1]:
!pip list | grep -iE "lang(smith|chain)"

langchain                                1.0.3
langchain-core                           1.0.3
langchain-groq                           1.0.0
langchain-text-splitters                 0.3.11
langsmith                                0.4.41


### Variables Required

| Token Name             | `.env` Name          | Where to Get / Setting Value                                                      |                                                                                              Reference |
| :--------------------- | :------------------- | :-------------------------------------------------------------------------------- | -----------------------------------------------------------------------------------------------------: |
| Groq API Key           | `GROQ_API_KEY`       | [Groq Console](https://console.groq.com/keys)                                     |                                              [Groq API Docs](https://console.groq.com/docs/quickstart) |
| LangSmith API Key      | `LANGSMITH_API_KEY`  | [LangSmith Settings](https://smith.langchain.com/settings)                        |                                   [LangSmith API Reference](https://docs.langchain.com/langsmith/home) |
| LangSmith Tracing      | `LANGSMITH_TRACING`  | A Boolean, set the value to `true` or `false` to enable or disable logging traces | [LangSmith Observability API Reference](https://docs.langchain.com/langsmith/observability-quickstart) |
| LangSmith Endpoint     | `LANGSMITH_ENDPOINT` | The LangSmith Endpoint to log the traces (<https://api.smith.langchain.com>)      | [LangSmith Observability API Reference](https://docs.langchain.com/langsmith/observability-quickstart) |
| LangSmith Project Name | `LANGSMITH_PROJECT`  | The name of the project to log the traces under                                   | [LangSmith Observability API Reference](https://docs.langchain.com/langsmith/observability-quickstart) |


## Actual Shenanigans


In [7]:
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_groq import ChatGroq
from langchain.messages import HumanMessage

from google.colab import userdata


### Defining Tools

Read the docs [here](https://reference.langchain.com/python/langchain/tools/)


In [3]:
@tool
def get_weather(city: str) -> str:
    """Get the weather for a given city."""
    # Mock data
    weather_data = {
        "San Francisco": "Sunny, 72°F",
        "New York": "Cloudy, 65°F",
        "London": "Rainy, 55°F",
        "Tokyo": "Sunny, 75°F",
    }
    return weather_data.get(city, f"Weather data for {city} not available")


@tool
def get_population(city: str) -> str:
    """Get the population of a city."""
    population_data = {
        "San Francisco": "873,965",
        "New York": "8,335,897",
        "London": "9,002,488",
        "Tokyo": "37,400,068",
    }
    return population_data.get(city, f"Population data for {city} not available")


@tool
def get_currency_exchange(from_currency: str, to_currency: str) -> str:
    """Get the currency exchange rate between two currencies."""
    exchange_rates = {
        ("USD", "EUR"): "0.85",
        ("EUR", "USD"): "1.18",
        ("USD", "JPY"): "110.0",
        ("JPY", "USD"): "0.0091",
    }
    return exchange_rates.get(
        (from_currency, to_currency),
        f"Exchange rate from {from_currency} to {to_currency} not available",
    )


@tool
def search_web(query: str) -> str:
    """Search the web for information."""
    # Mock search results
    return f"Search results for '{query}': Found 1.2M results (mocked)"


### Initializing the Model and the Agent

In [8]:
# !NOTE the GROQ_API_KEY must be set for this!

groq_model = ChatGroq(
    api_key=userdata.get('GROQ_API_KEY'),
    model="llama-3.3-70b-versatile",
    temperature=0.7,
)

agent = create_agent(
    model=groq_model,
    tools=[get_weather, get_population, get_currency_exchange, search_web],
    system_prompt="You are a helpful assistant that provides information about cities.",
)


### Running the Agent

#### Example 1: Simple Query

In [9]:
print("=" * 70)
print("EXAMPLE 1: Basic Query")
print("=" * 70)

result = agent.invoke(
    {"messages": [HumanMessage(content="What's the weather in San Francisco?")]}
)

print(result["messages"][-1].content)


EXAMPLE 1: Basic Query
The current weather in San Francisco is sunny with a temperature of 72°F.


#### Example 2: Complex Query Requiring Multiple Tools

In [10]:
print("=" * 70)
print("EXAMPLE 2: Multi-Tool Query")
print("=" * 70)

result = agent.invoke(
    {
        "messages": [
            HumanMessage(
                content="Tell me about New York - what's the weather and population?"
            )
        ]
    }
)

print(result["messages"][-1].content)


EXAMPLE 2: Multi-Tool Query
The current weather in New York is cloudy with a temperature of 65°F. The population of New York is approximately 8,335,897 people.


#### Example 3: Interactive Mode

In [ ]:
import ipywidgets as widgets
from IPython.display import display

text = widgets.Text(placeholder="Ask away!")
send = widgets.Button(description="Send")
out = widgets.Output()

print("=" * 70)
print("EXAMPLE 3: Interactive Mode")
print("=" * 70)


def on_click(b):
    query = text.value.strip()
    if not query:
        return
    text.value = ""  # clear input
    with out:
        print(f"> {query}")
        try:
            result = agent.invoke({"messages": [HumanMessage(content=query)]})
            print(result["messages"][-1].content)
        except Exception as e:
            print("Error:", e)


send.on_click(on_click)
display(widgets.VBox([text, send, out]))
